## ⚒️ Módulo 7 – Testes Automatizados com Frameworks Python (Pesquisa)

Embora o Selenium seja uma ferramenta de automação, seu uso mais comum é em **testes de software**. Para criar testes robustos, organizados e escaláveis, integramos o Selenium com frameworks de teste como `unittest` (nativo do Python) ou `pytest` (mais moderno e com uma sintaxe mais limpa).

Usaremos o **pytest** por sua simplicidade e poder.

### **Objetivos**
- Familiarização com o pyest e selenium
- Pesquisar como funciona a integração do pytest com selenium

link: https://docs.pytest.org/en/stable/


### **Estrutura de um Teste com Pytest**
1.  **Instalação:** `pip install pytest selenium`
2.  **Arquivo de Teste:** Crie um arquivo Python com o nome começando com `test_` (ex: `test_login.py`) ou terminando com `_test.py`.
3.  **Fixture de Setup/Teardown:** Uma função marcada com `@pytest.fixture` que prepara o ambiente (cria o `driver`) e o limpa no final (`yield` e depois `driver.quit()`).
4.  **Funções de Teste:** Funções que começam com `test_` e que recebem a fixture como argumento. Dentro delas, usamos `assert` para validar os resultados.
5.  **Execução:** No terminal, navegue até a pasta do projeto e execute o comando `pytest`.

### **Gabarito: Criar testes automatizados para o fluxo de login**
Vamos criar um arquivo de teste para o site ["The-Internet"](https://the-internet.herokuapp.com/login) com dois cenários de teste distintos:
1.  Teste de login com credenciais válidas.
2.  Teste de login com credenciais inválidas.

```python
# Salve este código em um arquivo chamado: test_login_flow.py   # comentário meta: nome do arquivo sugerido

import pytest  # importa o framework de testes pytest
from selenium import webdriver  # importa a API principal do Selenium para controlar o navegador
from selenium.webdriver.common.by import By  # enum para estratégias de localização (ID, CSS_SELECTOR, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait  # utilitário para esperas explícitas (aguardar condições)
from selenium.webdriver.support import expected_conditions as EC  # conjunto de condições prontas (elemento visível, clicável, etc.)

# 3. Fixture: Prepara o ambiente antes de cada teste e limpa depois.  # comentário explicativo do bloco
@pytest.fixture  # marca a função como fixture do pytest (injeta dependências em testes)
def driver():
    """Esta fixture inicializa o WebDriver antes de cada função de teste
    e o fecha após a execução do teste."""  # docstring: descreve o propósito da fixture
    # --- SETUP ---
    navegador = webdriver.Chrome()  # instancia um driver do Chrome (requer chromedriver compatível)
    navegador.get('https://the-internet.herokuapp.com/login')  # abre a URL de login (⚠️ ver observação sobre essa string)

    # A palavra 'yield' entrega o controle (e o objeto 'navegador') para a função de teste.
    yield navegador  # entrega o driver para o teste; o código após o yield roda no teardown

    # --- TEARDOWN ---
    # O código após o 'yield' é executado após o término de cada teste.
    print("Fechando o navegador...")  # log simples para depuração/observação
    navegador.quit()  # fecha o navegador e encerra a sessão do WebDriver

# 4. Função de Teste para o cenário de sucesso
def test_login_sucesso(driver):  # define um caso de teste; recebe a fixture 'driver'
    """Verifica se o login com credenciais válidas funciona e redireciona
    para a página segura."""  # docstring: descreve o objetivo do teste
    print("Executando teste de login com sucesso...")  # log opcional para rastrear execução
    driver.find_element(By.ID, 'username').send_keys('tomsmith')  # digita o usuário válido no campo #username
    driver.find_element(By.ID, 'password').send_keys('SuperSecretPassword!')  # digita a senha válida no campo #password
    driver.find_element(By.CSS_SELECTOR, 'button.radius').click()  # clica no botão de login localizado por seletor CSS

    # 5. Assertion: O momento da verificação (a alma do teste)
    wait = WebDriverWait(driver, 10)  # cria um esperador explícito com timeout de 10s
    mensagem_sucesso = wait.until(EC.visibility_of_element_located((By.ID, 'flash')))  # aguarda até a caixa de mensagem (#flash) ficar visível

    # Verificamos se a URL contém a palavra 'secure'
    assert 'secure' in driver.current_url  # valida se houve redirecionamento para /secure
    # Verificamos se a mensagem de sucesso é a esperada
    assert 'You logged into a secure area!' in mensagem_sucesso.text  # assegura que o texto de sucesso está presente

# 4. Função de Teste para o cenário de falha
def test_login_falha(driver):  # segundo caso de teste: credenciais inválidas
    """Verifica se o login com credenciais inválidas exibe uma mensagem de erro
    apropriada."""  # docstring: descreve o objetivo do teste
    print("Executando teste de login com falha...")  # log opcional
    driver.find_element(By.ID, 'username').send_keys('usuarioerrado')  # preenche usuário inválido
    driver.find_element(By.ID, 'password').send_keys('senhaerrada')  # preenche senha inválida
    driver.find_element(By.CSS_SELECTOR, 'button.radius').click()  # tenta autenticar

    # 5. Assertion: Verificamos se a mensagem de erro aparece
    wait = WebDriverWait(driver, 10)  # cria o esperador explícito
    mensagem_erro = wait.until(EC.visibility_of_element_located((By.ID, 'flash')))  # espera a mensagem de erro aparecer

    assert 'Your username is invalid!' in mensagem_erro.text  # valida que o texto de erro esperado está presente

# Para executar:
# 1. Salve este arquivo como 'test_login_flow.py'.
# 2. Abra o terminal na pasta onde o arquivo foi salvo.
# 3. Execute o comando: pytest -v
# (O -v significa 'verbose', para uma saída mais detalhada)  # instruções de execução via CLI
